## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [24]:
f.predictions('nvda')

NVDA 5m Interval Timestamp: 2024-11-08 04:29:26 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,static
kelly_1:2.5,0.135032,0.119917,0.158246,0.176361
prob_up,0.380886,0.355616,0.357401,0.25
prob_static,0.267787,0.313855,0.339744,0.5
prob_down,0.351327,0.330529,0.302855,0.25
precision,0.382166,0.371369,0.398747,0.411687
recall,0.337079,0.335206,0.357678,0.581614
f1,0.358209,0.352362,0.377098,0.482115
support,"[534.0, 533.0, 533.0]","[534.0, 533.0, 533.0]","[534.0, 533.0, 533.0]","[534.0, 533.0, 533.0]"


NVDA 15m Interval Timestamp: 2024-11-08 04:29:32 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,static
kelly_1:2.5,0.075,0.081967,0.103125,0.174672
prob_up,0.189683,0.31155,0.333335,0.375
prob_static,0.248408,0.32894,0.310346,0.375
prob_down,0.561909,0.35951,0.356319,0.25
precision,0.339286,0.344262,0.359375,0.41048
recall,0.320225,0.353933,0.38764,0.531073
f1,0.32948,0.34903,0.372973,0.463054
support,"[177.0, 177.0, 178.0]","[177.0, 177.0, 178.0]","[177.0, 177.0, 178.0]","[177.0, 177.0, 178.0]"


NVDA 1h Interval Timestamp: 2024-11-08 04:29:35 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,up
kelly_1:2.5,0.093933,0.129646,0.124598,0.115423
prob_up,0.328118,0.283695,0.341503,0.75
prob_static,0.385477,0.385809,0.343323,0.25
prob_down,0.286404,0.330496,0.315174,0.0
precision,0.352809,0.378319,0.374713,0.368159
recall,0.363426,0.395833,0.377315,0.342593
f1,0.358039,0.386878,0.376009,0.354916
support,"[432.0, 432.0, 432.0]","[432.0, 432.0, 432.0]","[432.0, 432.0, 432.0]","[432.0, 432.0, 432.0]"


NVDA 1wk Interval Timestamp: 2024-11-08 04:29:46 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,down,static
kelly_1:2.5,0.12093,0.008333,0.056842,0.0625
prob_up,0.219627,0.221084,0.296547,0.25
prob_static,0.420485,0.259717,0.299456,0.375
prob_down,0.359887,0.5192,0.403997,0.375
precision,0.372093,0.291667,0.326316,0.330357
recall,0.359551,0.314607,0.348315,0.41573
f1,0.365714,0.302703,0.336957,0.368159
support,"[89.0, 89.0, 89.0]","[89.0, 89.0, 89.0]","[89.0, 89.0, 89.0]","[89.0, 89.0, 89.0]"


NVDA 1mo Interval Timestamp: 2024-11-08 04:29:48 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,static,down,static
kelly_1:2.5,-0.008,0.133333,-0.023077,0.096774
prob_up,0.26903,0.373928,0.329675,0.375
prob_static,0.238755,0.476407,0.309909,0.375
prob_down,0.492215,0.149665,0.360416,0.25
precision,0.28,0.380952,0.269231,0.354839
recall,0.35,0.4,0.35,0.55
f1,0.311111,0.390244,0.304348,0.431373
support,"[19.0, 20.0, 20.0]","[19.0, 20.0, 20.0]","[19.0, 20.0, 20.0]","[19.0, 20.0, 20.0]"


###### Check Summary Output

In [23]:
symbol='NVDA'
interval='1h'

f.transform(symbol, interval)

df = f.load_model_df(symbol,interval)

df.tail(7)

,top_z7,body_z7,bottom_z7,vol_z7,pct_gap_up_down_z5,ac_z5,ac_z7,kma_sma7_diff_z5,sma_short_diff_z5,day_of_month,day_of_week,hour_of_day,cluster,direction
Datetime,,,,,,,,,,,,,,
2024-11-07 09:30:00-05:00,1.113708,0.413870,-1.036280,2.267787,0.285251,1.660555,2.026247,0.785008,-1.631969,7,3,9,0.0,0
2024-11-07 10:30:00-05:00,0.941270,-1.738618,1.554757,0.589526,0.486179,0.997242,1.292557,0.167544,-1.411373,7,3,10,0.0,0
2024-11-07 11:30:00-05:00,1.901669,-1.310810,0.504528,-0.014718,0.496389,0.716326,1.021568,-1.678113,-1.212700,7,3,11,0.0,2
2024-11-07 12:30:00-05:00,0.324989,-0.546399,0.525892,-0.087986,0.449999,0.181343,0.607142,-1.469021,-1.049878,7,3,12,0.0,0
2024-11-07 13:30:00-05:00,1.953196,-0.852922,-0.764985,-0.132165,0.305196,-0.420956,0.516458,-1.244747,-0.987151,7,3,13,0.0,1
2024-11-07 14:30:00-05:00,-0.643096,0.162776,0.364478,0.029761,1.320206,1.562456,0.780274,-1.133968,0.577784,7,3,14,0.0,1
2024-11-07 15:30:00-05:00,-1.272387,1.303214,-0.281204,-0.676758,-0.809225,1.579170,1.985234,-1.182412,1.421399,7,3,15,0.0,NaN


In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Select columns ending with 'z21'
z_columns = ['ac_z5', 'ac_z8', 'ac_z13', 'ac_z21', 'top_z21', 'body_z21', 'bottom_z21', 'vol_z21', 'pct_gap_up_down_z21', 'kma_sma40_diff_z21']
data_z21 = df[z_columns].dropna()  # Drop rows with missing values for simplicity

# Create an elbow plot to find the optimal number of clusters
inertia = []
cluster_range = range(1, 11)  # Test for clusters from 1 to 10

for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_z21)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Plot for Optimal Number of Clusters')
plt.xticks(cluster_range)
plt.grid(True)
plt.show()


In [ ]:
optimal_k = 3  # Replace with the optimal number from the elbow plot
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
data_z21['cluster'] = kmeans.fit_predict(data_z21)

# Check the cluster distribution
print(data_z21['cluster'].value_counts())

In [ ]:
# Add the 'cluster' column back to the original DataFrame
df['cluster'] = data_z21['cluster']



In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')